In [1]:
import timbermafia
import logging
import sys
lorem = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."

In [2]:
_valid_styles = ['default', 'test1']

from logging import root

divider_flag = 'divider_replace_me'

def divider():
    return divider_flag

def headed_log(func=None):
    """A decorator for header breaks in stdout."""

    def decorator_divider(self, *args, **kwargs):
        func(divider())
        func(*args, **kwargs)
        func(divider())

    return decorator_divider

def enhance(log):
    levels = []
    for level_name in logging._nameToLevel.keys():
        if level_name != 'NOTSET':
            levels.append(level_name.lower())
    levels = set(levels)
    funcs = [getattr(root, level) for level in levels]
    for func, level in zip(funcs, levels):
        print(func, level)
        setattr(log, f'h{level}', headed_log(func=func))

def configure_root_logger(*args, **kwargs):
    """Function to configure the root logger in logging, analagous to logging.basicConfig"""
    # Reset handlers
    force = kwargs.get('force', True)
    if force:
        for h in root.handlers[:]:
            root.removeHandler(h)
            h.close()
        
    # Pre-determined style
    style = kwargs.get('style', 'default')
    if style not in _valid_styles:
        raise ValueError(f'"style" must be one of: {", ".join(_valid_styles)}')
            
    ###################################################################################
    # Configure formatters
    ###################################################################################
    show_level = kwargs.get('show_level', False)
    user_format = kwargs.get('format', False)
    time_format = kwargs.get('time_format', '%H:%M:%S')
    format_style = kwargs.get('format_style', '{')
    
    my_format = user_format if user_format else '{asctime} | {name}.{funcName} | {message}'
    my_format_loglevel = user_format if user_format else '{asctime} | {levelname} | {name}.{funcName} | {message}'
    format_to_use = my_format_loglevel if show_level else my_format
    
    stream_formatter = timbermafia.TMFormatter(format_to_use, time_format,
                                               style=format_style)
    file_formatter = timbermafia.TMFormatter(my_format_loglevel, time_format,
                                             style=format_style)
    
    ###################################################################################
    # Configure handlers
    ###################################################################################
    handlers = []
    # Configure stream handler if required.
    stream = kwargs.get('stream', None)
    show_colour = kwargs.get('show_colour', True)
    if stream:
        if show_colour:
            s = timbermafia.RainbowStreamHandler(stream=stream)
        else:
            s = logging.StreamHandler(stream=stream)
        s.setFormatter(stream_formatter)
        handlers.append(s)
        
    # Configure file handler if required.
    filename = kwargs.get('filename', None)
    if filename:
        f = logging.FileHandler(filename)
        f.setFormatter(file_formatter)
        handlers.append(f)
        
    ###################################################################################
    # Final config
    ###################################################################################
    for h in handlers:
        root.addHandler(h)
    
    # Set level
    level = kwargs.get('level', 'DEBUG')
    root.setLevel(level)
    
    # Enhance the root log
    enhance(logging.Logger)
    
configure_root_logger(style='test1', stream = sys.stdout, filename='/tmp/test.log')

def getLogger(name):
    log = logging.getLogger(name)
    enhance(log)
    return log

<bound method Logger.warn of <RootLogger root (DEBUG)>> warn
<bound method Logger.critical of <RootLogger root (DEBUG)>> fatal
<bound method Logger.warning of <RootLogger root (DEBUG)>> warning
<bound method Logger.critical of <RootLogger root (DEBUG)>> critical
<bound method Logger.error of <RootLogger root (DEBUG)>> error
<bound method Logger.info of <RootLogger root (DEBUG)>> info
<bound method Logger.debug of <RootLogger root (DEBUG)>> debug


In [3]:
log = logging.getLogger('mylog')

def test_log():
    log.debug('test from func')

In [4]:
log.info(lorem)
log.info('This looks like a filename: /tmp/a_file.db')
log.info('This looks like a url: https://someaddress.com')

log.debug('a debug message')
log.warning('a warning message')
log.error('found error we can recover from')

log.fatal('found a fatal error')

# log.exception(ValueError('wrong value'), exc_info=True)

19:17:40 |                     mylog | Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod
         |                           | tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam,
         |                           | quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo
         |                           | consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse
         |                           | cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non
         |                           | proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
19:17:40 |                     mylog | This looks like a filename: /tmp/a_file.db
19:17:40 |                     mylog | This looks like a url: https://someaddress.com
19:17:40 |                     mylog | a debug message
19:17:40 |                     mylog | a warning message
19:17:40 |                     mylog | f

In [5]:
# enhance(log)
log.hinfo('a title')
log.info(lorem)
log.hwarning('title with warning')
log.hdebug('title with debug')
log.herror('error title')
print('white')

------------------------------------------------------------------------------------------------------------------------
19:17:40 |                           | a title
------------------------------------------------------------------------------------------------------------------------
19:17:40 |                     mylog | Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod
         |                           | tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam,
         |                           | quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo
         |                           | consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse
         |                           | cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non
         |                           | proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
-------------------------------

white


In [6]:
root.hinfo('test@')

------------------------------------------------------------------------------------------------------------------------
19:17:40 |                           | test@
------------------------------------------------------------------------------------------------------------------------


In [7]:
test_log()

19:17:40 |            mylog.test_log | test from func


In [8]:
class Template(timbermafia.Logged):
    pass

    def some_method(self):
        self.log.debug('output from a class method')

A = Template()
A.log.info('Test logging coming from inside a Template class')
A.some_method()

19:17:40 |                  Template | Test logging coming from inside a Template class
19:17:40 |      Template.some_method | output from a class method


In [9]:
log.error('another error')

19:17:40 |                     mylog | another error


In [10]:
RED = "\033[38;5;214m"
print(RED + 'some red text')

some red text
